<a href="https://colab.research.google.com/github/F1ameX/Modern-Methods-of-Deep-Machine-Learning/blob/main/2_elementary_perceptron/2_elementary_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from numpy.typing import ArrayLike
from typing import Optional
import matplotlib.pyplot as plt

In [40]:
def make_circles(n_samples : int = 100,
                 shuffle : bool = True,
                 noise : float = None,
                 random_state : int = None,
                 factor : float = 0.8):

    rng = np.random.default_rng(seed = random_state)

    n_samples_out = n_samples // 2
    n_samples_in = n_samples - n_samples_out

    linspace_out = np.linspace(0, 2 * np.pi, n_samples_out, endpoint = False)
    linspace_in = np.linspace(0, 2 * np.pi, n_samples_in, endpoint = False)

    X1_out = np.cos(linspace_out)
    X2_out = np.sin(linspace_out)

    X1_in = np.cos(linspace_in) * factor
    X2_in = np.sin(linspace_in) * factor

    X = np.vstack(
        [np.append(X1_out, X1_in), np.append(X2_out, X2_in)]
    ).T

    y = np.hstack (
        [np.zeros(n_samples_out, dtype = int), np.ones(n_samples_in, dtype = int)]
    )

    if shuffle:
        permutation = rng.permutation(n_samples)
        X = X[permutation]
        y = y[permutation]

    if noise is not None:
        X += rng.normal(scale = noise, size = X.shape)

    return X, y


def make_blobs(n_samples : int = 100,
               n_features : int = 2,
               centers : int | ArrayLike | None = None,
               cluster_std : float | ArrayLike = 1.0,
               center_box : tuple[float, float] = (-10.0, 10.0),
               shuffle : bool = True,
               random_state : int = 42,
               return_centers : bool = False):

    rng = np.random.default_rng(seed = random_state)

    if centers is None:
        n_centers = 2

        centers = rng.uniform(
            center_box[0], center_box[1], size = (n_centers, n_features)
        )

    elif isinstance(centers, int):
        n_centers = centers
        centers = rng.uniform(
            center_box[0], center_box[1], size = (n_centers, n_features)
        )

    else:
        centers = np.asarray(centers, dtype = float)
        n_centers = centers.shape[0]

    base = n_samples // n_centers
    extra = n_samples % n_centers

    clusters = np.zeros(n_centers, dtype = int)

    for k in range(n_centers):
        if k < extra:
            clusters[k] = base + 1
        else:
            clusters[k] = base

    if isinstance(cluster_std, int | float):
        cluster_std_sigma = cluster_std
        cluster_std = np.repeat(cluster_std_sigma, n_centers)

    X = np.empty((n_samples, n_features))
    y = np.empty(n_samples, dtype = int)

    pos = 0

    for k in range(n_centers):
        normal_matrix = rng.normal(loc = 0, scale = 1, size = (clusters[k], n_features))
        X_k = centers[k] + cluster_std[k] * normal_matrix

        y[pos : pos + clusters[k]] = k
        X[pos : pos + clusters[k]] = X_k
        pos += clusters[k]

    if shuffle:
        permutation = rng.permutation(n_samples)
        X = X[permutation]
        y = y[permutation]

    if return_centers:
        return X, y, centers

    return X, y

In [41]:
make_blobs()

(array([[ 5.54515167, -0.09519   ],
        [ 5.59580678, -1.00374261],
        [ 6.09510039, -0.09345891],
        [ 7.50953295,  5.35484244],
        [ 8.10353141,  4.33231155],
        [ 9.16868929,  6.86122305],
        [ 9.30042813,  3.1259739 ],
        [ 6.5598616 ,  3.80656969],
        [ 5.122857  , -0.48491564],
        [ 8.77373729,  3.70800495],
        [ 5.62154671, -0.53194585],
        [ 5.05582266, -1.30214942],
        [ 6.57780844,  2.50130273],
        [ 5.00874832, -1.86130905],
        [ 6.23834072,  3.74192302],
        [ 6.7821486 ,  2.57067443],
        [ 7.30223292,  4.93010009],
        [ 6.70166231, -1.37696069],
        [ 7.62076857, -1.62884622],
        [ 5.46231981, -2.07547513],
        [ 5.05186832, -1.06389151],
        [ 5.10095842,  0.07679709],
        [ 5.20397872,  0.27251011],
        [ 3.79178654, -2.66954368],
        [ 3.7962512 , -1.55731623],
        [ 7.81528519,  3.55275546],
        [ 5.76824037, -0.59114298],
        [ 5.49727545,  3.461